Mar 1

Measure and plot precision, recall and F1

Mar 3

The current model concatenates the sentence mixes before feeding
them into the linear layer. This is wrong as it does not assign a
class embedding to a single output class.

Instead, each sentence mix should be passed through the linear
layer individually.

# Imports

In [1]:
%load_ext autoreload
%autoreload 2

from datetime import datetime
from typing import List, Tuple

import pandas as pd
import torch
from IPython.core.display import display
from sklearn.metrics import precision_score, recall_score, f1_score
from torch import tensor, Tensor
from torch.nn import BCEWithLogitsLoss
from torch.optim import Adam
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from torchtext.data import Field, TabularDataset

from dao.ower.ower_dir import Sample
# from base_classifier import Classifier
# from attention_classifier import Classifier
from classifier import Classifier

pd.set_option('display.max_colwidth', None)
pd.set_option('display.precision', 2)

# Config

In [2]:
ower_dataset = 'ower-v3-fb-irt-3'
class_count = 4
sent_count = 3

# vectors = None
# vectors = 'charngram.100d'
# vectors = 'fasttext.en.300d'
# vectors = 'fasttext.simple.300d'
# vectors = 'glove.42B.300d'
# vectors = 'glove.840B.300d'
# vectors = 'glove.twitter.27B.25d'
# vectors = 'glove.twitter.27B.50d'
# vectors = 'glove.twitter.27B.100d'
vectors = 'glove.twitter.27B.200d'
# vectors = 'glove.6B.50d'
# vectors = 'glove.6B.100d'
# vectors = 'glove.6B.200d'
# vectors = 'glove.6B.300d'

emb_size = None
# emb_size = 200

batch_size = 1024
sent_len = 64

class_weight = 4
lr = 0.1
epoch_count = 50

log_dir = 'runs/' + datetime.now().strftime("%Y-%m-%d_%H-%M-%S") + \
          f'_{ower_dataset}' + f'_emb-{emb_size}'

# 1 Build train/valid DataLoaders

## 1.1 Read Sample TSVs into TabularDatasets

In [3]:
#
# Define columns for subsequent read into TabularDatasets
#

def tokenize(text: str) -> List[str]:
    return text.split()

raw_ent_field = Field(sequential=False, use_vocab=False)
raw_class_field = Field(sequential=False, use_vocab=False)
raw_sent_field = Field(sequential=True, use_vocab=True, tokenize=tokenize, lower=True)

ent_field = ('ent', raw_ent_field)
class_fields = [(f'class_{i}', raw_class_field) for i in range(class_count)]
sent_fields = [(f'sent_{i}', raw_sent_field) for i in range(sent_count)]

fields = [ent_field] + class_fields + sent_fields

#
# Read Train Samples TSV into TabularDataset
#

train_samples_tsv = f'data/{ower_dataset}/train.tsv'
valid_samples_tsv = f'data/{ower_dataset}/valid.tsv'

raw_train_set = TabularDataset(train_samples_tsv, 'tsv', fields, skip_header=True)
raw_valid_set = TabularDataset(valid_samples_tsv, 'tsv', fields, skip_header=True)

#
# Print some samples
#

tab_cols = ['ent'] + [f'class_{i}' for i in range(class_count)] + [f'sent_{i}' for i in range(sent_count)]

train_tab_data = [[getattr(row, col) for col in tab_cols] for row in raw_train_set[:20]]
display(pd.DataFrame(train_tab_data, columns=tab_cols))

valid_tab_data = [[getattr(row, col) for col in tab_cols] for row in raw_valid_set[:20]]
display(pd.DataFrame(valid_tab_data, columns=tab_cols))

c:\users\tobias\documents\hsrm\master\thesis\thesis-tools\venv\lib\site-packages\torchtext\data\field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
c:\users\tobias\documents\hsrm\master\thesis\thesis-tools\venv\lib\site-packages\torchtext\data\example.py:68: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)
c:\users\tobias\documents\hsrm\master\thesis\thesis-tools\venv\lib\site-packages\torchtext\data\example.py:78: UserWarning: Example clas

,ent,class_0,class_1,class_2,class_3,sent_0,sent_1,sent_2
0,1,0,0,0,0,"[the, republic, of, minerva, was, a, micronation, consisting, of, the, minerva, reefs.]","[the, belarusian, people's, republic, was, declared, on, the, german-occupied, territory, of, modern-day, belarus, three, weeks, after, the, treaty, of, brest-litovsk, was, signed, on, march, 3,, 1918, between, the, new, bolshevik, government, of, soviet, russia, and, the, central, powers, in, the, border, city, of, brest-litovsk.]","[some, of, the, action, in, joe, haldeman's, 1989, science, fiction, novel, buying, time, (published, in, uk, as, the, long, habit, of, living), takes, place, in, the, conch, republic,, a, lawless, place, where, assassination, and, other, activities, are, perfectly, legal.]"
1,2,0,0,0,0,"[elements, from, dairanger, were, adapted, into, the, second, season, of, mighty, morphin, power, rangers,, specifically, the, action, sequences, between, the, giant, robots, (which, became, the, power, rangers', thunderzords), and, some, of, the, monsters.]","[a, different, drum"", kimberly, is, revealed, to, be, proficient, in, many, hobbies, throughout, the, series,, including, designing, and, arranging, the, floral, pattern, for, a, parade, float,mighty, morphin, power, rangers;, ""the, spit, flower"", being, regarded, as, a, great, cheerleader, throughout, junior, high, schoolmighty, morphin, power, rangers;, ""enter..., the, lizzinator"", and, being, able, to, sing, and, play, guitar.]","[at, age, 14,, he, appeared, in, the, series, mighty, morphin, power, rangers, as, an, uncredited, extra, in, the, episode, ""second, chance"".]"
2,4,0,0,0,0,"[itzkoff, wrote, that, critics, fear, that, ""rape, has, become, so, pervasive, in, the, drama, that, it, is, almost, background, noise:, a, routine, and, unshocking, occurrence"".]","[pariyerum, perumal, (), is, a, 2018, indian, tamil, language, drama, film, produced, by]","[a, free, soul, is, a, 1931, american, pre-code, drama, film, that, tells, the, story, of, an, alcoholic, san, francisco, defense, attorney, who, must, defend, his, daughter's, ex-boyfriend, on, a, charge, of, murdering, the, mobster, she, had, started, a, relationship, with,, who, he, had, previously, gotten, an, acquittal, for, on, a, murder, charge.]"
3,5,0,0,0,0,"[balk, also, appeared, in, valmont,, the, craft,, the, island, of, dr., moreau,, american, history, x,, the, waterboy,, almost, famous,, and, personal, velocity:, three, portraits.]","[the, anti-heros, were, involved, in, a, lawsuit, against, new, line, cinema, for, using, the, band's, logo, in, the, film, american, history, x, against, the, band's, will, (as, they, did, not, want, to, be, associated, with, white, supremacists).]","[adrien, brody, narrated, the, film, with, esai, morales;, prior, to, that, he, received, critical, acclaim, for, his, feature, film, intermedio,, a, supernatural, thriller, starring, edward, furlong, (terminator, 2:, judgment, day,, american, history, x), and, amber, benson, (buffy, the, vampire, slayer), and, the, multi-award-winning, short, film,, little, cuba.]"
4,6,0,0,1,1,"[delancer, got, his, start, as, a, writer,, researcher,, journalist,, documentarian, and, television, producer, in, the, dominican, republic, throughout, the, 1980s, and, 1990s, before, beginning, work, on, his, first, feature, film, debut,, trópico, de, sangre,, which, tells, the, story, of, the, mirabal, sisters, in, times, of, rafael, trujillo, dictatorship,, and, is, co-produced, by, and, starring, actress, michelle, rodriguez.]","[the, third-person, action,, which, acts, as, a, prequel, to, the, film,, features, sigourney, weaver,, stephen, lang,, michelle, rodriguez,, and, giovanni, ribisi, reprise, their, roles, from, the, film.]","[control, is, a, 2004, american, direct-to-video, film, directed, by, tim, hunter, and, starring, ray, liotta,, willem, dafoe, and, michelle, rodriguez.]"
5,7,1,0,1,1,"[widmore, pledges, to, help, locke, reunite, the, oceanic, six—jack, shep

,ent,class_0,class_1,class_2,class_3,sent_0,sent_1,sent_2
0,8208,0,0,0,0,"[at, one, new, york, show, they, were, discovered, by, an, a&m, records, talent, scout,, patrick, clifford,, and, the, band, signed, their, first, recording, contract.]","[although, this, song, entered, the, country, charts,, the, album, itself, was, not, released, due, to, the, closure, of, polydor/a&m's, nashville, unit.]","[the, song, gained, success, shortly, after, styx, left, wooden, nickel, records, to, move, to, a&m, records, in, 1974, as, it, began, picking, up, airplay, nationwide,styx, at, the, 2010, great, jones, county, fair, eventually, peaking, at, #6, on, the, billboard, hot, 100, in, march, 1975.]"
1,8209,0,0,1,1,"[after, discovering, he, is, an, unsuitable, donor, by, dr., leo, spaceman, (chris, parnell),, jack, uses, his, contacts—musicians, clay, aiken,, sara, bareilles,, the, beastie, boys, (mike, d, and, ad-rock),, mary, j., blige,, elvis, costello,, sheryl, crow,, steve, earle,, wyclef, jean,, norah, jones,, talib, kweli,, cyndi, lauper,, adam, levine,, michael, mcdonald,, rhett, miller,, moby,, robert, randolph,, and, rachael, yamagata—to, arrange, a, charity, benefit, entitled, ""kidney, now!""]","[her, compilation, and, specialty, albums, include, sheryl, crow, and, friends:, live, from, central, park, (1999),, the, very, best, of, sheryl, crow, (2003),, live, at, budokan, (2003),, itunes, originals, –, sheryl, crow, (2006), and, hits, &, rarities, (2007).]","[on, november, 7,, 1997,, after, repeating, the, song, ""a, change, would, do, you, good"", by, sheryl, crow, for, an, hour,, knrx, became, soft, adult, contemporary, kqsr,, seeking, to, target, the, 35–44, audience, and, citing, the, duplication, of, alternative, music, on, stations, in, other, formats, in, oklahoma, city.]"
2,8215,0,0,0,0,"[films, in, competition, included, babel, by, alejandro, gonzález, iñárritu,, fast, food, nation, by, richard, linklater,, iklimler, by, nuri, bilge, ceylan,, el, laberinto, del, fauno, by, guillermo, del, toro,, marie, antoinette, by, sofia, coppola,, southland, tales, by, richard, kelly,, volver, by, pedro, almodóvar,, and, the, wind, that, shakes, the, barley, by, ken, loach]","[inspired, by, pedro, almodóvar's, volver,, kim, began, writing, a, new, screenplay, in, 2008,, finished, its, first, draft, in, 2009,, and, continued, to, edit, it, until, 2013, when, filming, finally, went, underway.]","[alberto, iglesias, biography,, allmusic, iglesias, has, composed, a, number, of, scores, for, films, directed, by, pedro, almodóvar,, such, as, the, flower, of, my, secret, (1995),, live, flesh, (1997),, all, about, my, mother, (1999),, talk, to, her, (2002),, bad, education, (2004),, volver, (2006),, broken, embraces, (2009), and, the, skin, i, live, in, (2011).]"
3,8228,1,0,1,0,"[it, was, produced, by, tnt,, with, kyra, sedgwick,, series, creator, rob, bragin,, tom, jacobson,, jill, littman,, and, alex, graves, serving, as, executive, producers.]","[the, crude, oasis, is, a, 1995, american, drama, film, written, and, directed, by, alex, graves.]","[it, was, written, by, showrunners, and, executive, producers, david, benioff, and, d., b., weiss, and, directed, by, alex, graves,, his, directorial, debut, for, the, series.]"
4,8235,0,0,0,0,"[he, played, his, freshman, season, for, pappy, waldorf's, california, golden, bears, before, abruptly, transferring, to, ucla, in, the, fall, of, 1954.]","[this, is, seen, as, the, turning, point, for, both, ucla, and, usc's, football, programs.]","[in, his, senior, season,, he, recorded, 41, total, tackles, while, collecting, 3.5, sacks, for, the, bruins, which, propelled, him, to, win, the, ucla, kenneth, s., washington, award, for, outstanding, senior.]"
5,8237,0,0,0,0,"[released, exclusively, through, rough, trade, records,, the, single, was, limited, to, 300, copies, worldwide.]","[it, also, gave, them, control, of, the, established, music, company, rough, trade, (80%, of, rough, trade, records, germ

## 1.2 Build vocab on train data

In [4]:
raw_sent_field.build_vocab(raw_train_set, vectors=vectors)
vocab = raw_sent_field.vocab

#
# Print some samples
#

vocab_size = len(vocab)

print(vocab_size)
print(vocab.itos[:10])
print(vocab.itos[vocab_size//2:vocab_size//2+10])
print(vocab.itos[-10:])

112490
['<unk>', '<pad>', 'the', 'and', 'in', 'of', 'a', 'to', 'for', 'was']
['barsi.', 'barsoom,', 'barstow', "bart's", 'bart),', 'bartczak', 'bartell', 'bartender),', 'bartender,', 'barth,']
['中国电影史上第一位金鸡、百花的双料影后', '今夜不設防', '曹轩领4内援2外援加盟', '曾三度临危受命', '艳光四射)', '艷光四射,', '英雄悲情', "베라,'전북의", '새', '조형익;']


## 1.3 Transfor each TabularDataset -> List[Sample]

Parse texts from datasets, map words -> tokens (IDs) using vocab

In [5]:


def transform(raw_set: TabularDataset) -> List[Sample]:
    return [Sample(
        int(getattr(row, 'ent')),
        [int(getattr(row, f'class_{i}')) for i in range(class_count)],
        [[vocab[token] for token in getattr(row, f'sent_{i}')] for i in range(sent_count)]
    ) for row in raw_set]


train_set = transform(raw_train_set)
valid_set = transform(raw_valid_set)

#
# Print some samples
#

tab_cols = ['ent', 'classes', 'sents']
tab_data = [[getattr(row, col) for col in tab_cols] for row in train_set[:3]]

pd.DataFrame(tab_data, columns=tab_cols)

,ent,classes,sents
0,1,"[0, 0, 0, 0]","[[2, 919, 5, 15947, 9, 6, 86274, 2253, 5, 2, 15947, 96172], [2, 14646, 2588, 919, 9, 2633, 12, 2, 72600, 2429, 5, 15960, 12577, 87, 1743, 36, 2, 3233, 5, 58889, 9, 214, 12, 159, 624, 5334, 114, 2, 27, 58198, 433, 5, 1652, 2797, 3, 2, 348, 3022, 4, 2, 1666, 88, 5, 58891], [142, 5, 2, 426, 4, 352, 74426, 1111, 296, 551, 277, 11087, 96, 17113, 4, 771, 10, 2, 469, 15413, 5, 82959, 1369, 247, 4, 2, 17987, 7880, 6, 24799, 247, 60, 5103, 3, 55, 2823, 45, 25860, 35901]]"
1,2,"[0, 0, 0, 0]","[[1296, 18, 64809, 32, 644, 77, 2, 91, 109, 5, 3298, 13481, 768, 11888, 3452, 2, 426, 5928, 114, 2, 3397, 9432, 1229, 98, 2, 768, 95585, 105812, 3, 142, 5, 2, 13475], [6, 546, 67298, 15682, 14, 1501, 7, 51, 16261, 4, 102, 75999, 600, 2, 336, 42, 5768, 3, 10983, 2, 70783, 11786, 8, 6, 7196, 70762, 13481, 768, 38781, 228, 26995, 70814, 106, 2944, 10, 6, 327, 14825, 600, 1252, 153, 99276, 13481, 768, 38781, 43176, 2, 82972, 3, 106, 1510, 7, 3129, 3, 188, 8325], [16, 498, 1120, 15, 104, 4, 2, 48, 3298, 13481, 768, 3213, 10, 21, 4036, 3278, 4, 2, 185, 28203, 61167]]"
2,4,"[0, 0, 0, 0]","[[78342, 306, 20, 495, 3639, 20, 44207, 30, 462, 425, 38083, 4, 2, 126, 20, 23, 14, 1013, 2024, 37438, 6, 10578, 3, 108012, 90105], [91691, 38081, 940, 14, 6, 986, 478, 2353, 491, 126, 19, 112, 11], [6, 747, 1504, 14, 6, 4814, 39, 7214, 126, 19, 20, 2544, 2, 226, 5, 21, 7436, 248, 1581, 2031, 2372, 28, 2160, 11252, 17, 22959, 18382, 12, 6, 2622, 5, 10383, 2, 25323, 31, 35, 344, 6, 1143, 7347, 28, 15, 35, 395, 13075, 21, 52283, 8, 12, 6, 1339, 17863]]"


## 1.4 Build DataLoaders

In [6]:
def generate_batch(batch: List[Sample]) -> Tuple[Tensor, Tensor]:

    _ent, classes_batch, sents_batch = zip(*batch)

    cropped_sents_batch = [[sent[:sent_len]
                            for sent in sents] for sents in sents_batch]

    padded_sents_batch = [[sent + [0] * (sent_len - len(sent))
                           for sent in sents] for sents in cropped_sents_batch]

    return tensor(padded_sents_batch), tensor(classes_batch)


train_loader = DataLoader(train_set, batch_size=batch_size, collate_fn=generate_batch, shuffle=True)
valid_loader = DataLoader(valid_set, batch_size=batch_size, collate_fn=generate_batch)

# 2 Create classifier

In [7]:
if vocab.vectors is None:
    classifier = Classifier.from_random(len(vocab), emb_size, class_count)
else:
    classifier = Classifier.from_pre_trained(vocab, class_count)

print(classifier.class_embs)

Parameter containing:
tensor([[ 1.0378e+00, -5.0475e-01, -4.1587e-01, -1.4040e+00,  1.6143e+00,
          2.3962e+00,  6.7688e-01, -8.5163e-01,  1.2084e+00,  5.9328e-01,
          1.1053e+00,  6.9974e-02,  3.0069e-01,  1.5915e+00, -5.5176e-01,
         -1.2927e+00, -6.1994e-01,  1.4410e+00,  6.5129e-01, -1.1712e+00,
          7.7813e-01, -5.4101e-01,  2.5109e+00,  5.5882e-01, -1.1565e+00,
         -1.0947e+00,  1.0778e+00, -1.3565e+00,  8.2739e-01,  6.9359e-01,
          2.0125e+00,  1.0376e+00, -9.1750e-01,  6.5630e-01,  4.0354e-01,
         -5.6986e-01,  3.1507e-01, -4.0581e-01, -5.9916e-01,  5.6089e-01,
         -9.6712e-02,  7.5115e-01,  8.5681e-01, -1.3537e+00,  1.4678e+00,
          1.0888e-01, -9.1480e-01, -1.3673e+00,  1.8284e-01,  1.7150e+00,
          6.2555e-01,  9.1024e-01,  5.1454e-01,  1.4465e+00,  2.9358e-01,
          9.8779e-01,  1.0275e+00, -7.2989e-02,  7.5590e-01,  9.1872e-02,
         -7.2256e-01, -2.6189e-02,  4.5524e-01, -1.2802e+00,  1.2363e+00,
         -1.4313

# 3 Training

In [9]:
# criterion = MSELoss()
criterion = BCEWithLogitsLoss()
# criterion = BCEWithLogitsLoss(pos_weight=torch.tensor([class_weight] * class_count))

# optimizer = SGD(classifier.parameters(), lr=lr)
optimizer = Adam(classifier.parameters(), lr=lr)

writer = SummaryWriter(log_dir=log_dir)


for epoch in range(epoch_count):

    #
    # Train
    #

    # Train loss across all batches
    train_loss = 0.0

    # Valid gt/pred classes across all batches
    train_gt_classes_stack: List[List[int]] = []
    train_pred_classes_stack: List[List[int]] = []

    for batch_idx, (sents_batch, gt_classes_batch) in enumerate(train_loader):
        logits_batch = classifier(sents_batch)

        # loss = criterion(logits_batch, gt_classes_batch.float())
        loss = criterion(logits_batch, gt_classes_batch.float())
        train_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        pred_classes_batch = (logits_batch > 0).int()

        train_gt_classes_stack += gt_classes_batch.numpy().tolist()
        train_pred_classes_stack += pred_classes_batch.numpy().tolist()
        
        #
        # Print first batch
        #

        if batch_idx == 0:

            dlb = logits_batch.detach().numpy()  # logits batch
            dpb = pred_classes_batch.detach().numpy()  # predicted classes batch
            dgb = gt_classes_batch.detach().numpy()  # ground truth classes batch
            dsb = sents_batch.detach().numpy()  # sentences batch

            df_cols = ['entity', 'logits', 'p', 'gt', 'sents']
            df_data = [('foo', logits, pred_classes, classes, [[vocab.itos[tok] for tok in sent] for sent in sents])
                       for logits, pred_classes, classes, sents in zip(dlb, dpb, dgb, dsb)]

            df = pd.DataFrame(df_data, columns=df_cols)
            display(df)

    #
    # Validate
    #

    # Valid loss across all batches
    valid_loss = 0.0

    # Valid gt/pred classes across all batches
    valid_gt_classes_stack: List[List[int]] = []
    valid_pred_classes_stack: List[List[int]] = []

    with torch.no_grad():
        for batch_idx, (sents_batch, gt_classes_batch) in enumerate(valid_loader):
            logits_batch = classifier(sents_batch)

            # loss = criterion(logits_batch, gt_classes_batch.float())
            loss = criterion(logits_batch, gt_classes_batch.float())
            valid_loss += loss.item()

            pred_classes_batch = (logits_batch > 0).int()

            valid_gt_classes_stack += gt_classes_batch.numpy().tolist()
            valid_pred_classes_stack += pred_classes_batch.numpy().tolist()

            #
            # Print first batch
            #

            if batch_idx == 0:

                dlb = logits_batch.detach().numpy()  # logits batch
                dpb = pred_classes_batch.detach().numpy()  # predicted classes batch
                dgb = gt_classes_batch.detach().numpy()  # ground truth classes batch
                dsb = sents_batch.detach().numpy()  # sentences batch

                df_cols = ['entity', 'logits', 'p', 'gt', 'sents']
                df_data = [('foo', logits, pred_classes, classes, [[vocab.itos[tok] for tok in sent] for sent in sents])
                           for logits, pred_classes, classes, sents in zip(dlb, dpb, dgb, dsb)]

                df = pd.DataFrame(df_data, columns=df_cols)
                display(df)

    #
    # Log
    #

    print(f'Epoch {epoch}')

    train_loss /= len(train_loader)
    valid_loss /= len(valid_loader)

    print(f'    Loss:  train = {train_loss:.2f}, valid = {valid_loss:.2f}')
    writer.add_scalars('loss', {'train': train_loss, 'valid': valid_loss}, epoch)

    # tps = train precisions, vps = valid precisions, etc.
    tps = precision_score(train_gt_classes_stack, train_pred_classes_stack, average=None)
    vps = precision_score(valid_gt_classes_stack, valid_pred_classes_stack, average=None)
    trs = recall_score(train_gt_classes_stack, train_pred_classes_stack, average=None)
    vrs = recall_score(valid_gt_classes_stack, valid_pred_classes_stack, average=None)
    tfs = f1_score(train_gt_classes_stack, train_pred_classes_stack, average=None)
    vfs = f1_score(valid_gt_classes_stack, valid_pred_classes_stack, average=None)

    for i, (tp, vp, tr, vr, tf, vf) in enumerate(zip(tps, vps, trs, vrs, tfs, vfs)):
        print(f'    Precision {i}:  train = {tp:.2f}, valid = {vp:.2f}')
        print(f'    Recall {i}:  train = {tr:.2f}, valid = {vr:.2f}')
        print(f'    F1 {i}:  train = {tf:.2f}, valid = {vf:.2f}')
        writer.add_scalars('precision', {f'train {i}': tp, f'valid {i}': vp}, epoch)
        writer.add_scalars('recall', {f'train {i}': tr, f'valid {i}': vr}, epoch)
        writer.add_scalars('f1', {f'train {i}': tf, f'valid {i}': vf}, epoch)

    mean_train_precision = tps.mean()
    mean_valid_precision = vps.mean()
    mean_train_recall = trs.mean()
    mean_valid_recall = vrs.mean()
    mean_train_f1 = tfs.mean()
    mean_valid_f1 = vfs.mean()

    print(f'    Precision:  train = {mean_train_precision:.2f}, valid = {mean_valid_precision:.2f}')
    print(f'    Recall:  train = {mean_train_recall:.2f}, valid = {mean_valid_recall:.2f}')
    print(f'    F1:  train = {mean_train_f1:.2f}, valid = {mean_valid_f1:.2f}')
    writer.add_scalars('precision', {f'train': mean_train_precision, f'valid': mean_valid_precision}, epoch)
    writer.add_scalars('recall', {f'train': mean_train_recall, f'valid': mean_valid_recall}, epoch)
    writer.add_scalars('f1', {f'train': mean_train_f1, f'valid': mean_valid_f1}, epoch)

,entity,logits,p,gt,sents
0,foo,"[-3.6807883, -3.8685064, -3.2003095, -3.5692286]","[0, 0, 0, 0]","[0, 0, 0, 0]","[[a, product, of, real, madrid's, youth, system,, madrid-born, adrián, spent, 2006–07, with, the, reserves, real, madrid, castilla, where, he, was, trained, by, his, father,, former, club, legend, míchel., <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>], [born, in, las, arenas,, biscay,, zárraga, signed, for, real, madrid, in, 1949,, spending, the, following, two, seasons, with, the, reserve, team., <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>], [according, to, galán,, ""no, one, who, has, anything, to, do, with, the, football, world, can, be, unaware, that, zidane, is, acting, as, real, madrid, castilla's, head, coach, this, season., <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>]]"
1,foo,"[-1.2736795, -1.5042723, -1.2997435, -1.4568764]","[0, 0, 0, 0]","[1, 0, 1, 0]","[[on, february, 2,, 2011,, mgm, named, jonathan, glickman, to, be, the, film, president, of, mgm., <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>], [milton, glickman,, father, of, former, us, secretary, of, agriculture, dan, glickman, and, grandfather, of, film, producer, jonathan, glickman,, owned, the, team, from, 1970-1984., <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>], [in, 1993,, jonathan, glickman,, who, came, from, the, usc's, peter, stark, program,, joined, caravan, as, an, intern., <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>]]"
2,foo,"[-0.578886, -0.5959625, -1.9262898, -1.5232202]","[0, 0, 0, 0]","[0, 0, 0, 0]","[[kelly, oechsli, (1918, february, 23, –, 1999, october, 18), was, an, american, illustrator, (and, occasionally, author), of, children's, books., <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>], [the, presence:, a, ghost, story, is, a, children's, ghost, novel, by, eve, bunting., <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>], [in, 2008, the, award, was, renamed, ""best, children's, novel"", and, in, 2010, was, renam

,entity,logits,p,gt,sents
0,foo,"[-2.6785111, -3.4918373, -2.5617228, -4.032995]","[0, 0, 0, 0]","[0, 0, 0, 0]","[[at, one, new, york, show, they, were, discovered, by, an, a&m, records, talent, scout,, patrick, clifford,, and, the, band, signed, their, first, recording, contract., <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>], [although, this, song, entered, the, country, charts,, the, album, itself, was, not, released, due, to, the, closure, of, <unk>, nashville, unit., <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>], [the, song, gained, success, shortly, after, styx, left, wooden, nickel, records, to, move, to, a&m, records, in, 1974, as, it, began, picking, up, airplay, <unk>, at, the, 2010, great, jones, county, fair, eventually, peaking, at, #6, on, the, billboard, hot, 100, in, march, 1975., <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>]]"
1,foo,"[0.22008175, -0.6246566, 2.0078704, -3.3848429]","[1, 0, 1, 0]","[0, 0, 1, 1]","[[after, discovering, he, is, an, unsuitable, donor, by, dr., leo, <unk>, (chris, <unk>, jack, uses, his, <unk>, clay, aiken,, sara, bareilles,, the, beastie, boys, (mike, d, and, <unk>, mary, j., blige,, elvis, costello,, sheryl, crow,, steve, earle,, wyclef, jean,, norah, jones,, talib, kweli,, cyndi, lauper,, adam, levine,, michael, mcdonald,, rhett, miller,, moby,, robert, randolph,, and, rachael, <unk>, arrange, a, charity, benefit, entitled, <unk>, <unk>], [her, compilation, and, specialty, albums, include, sheryl, crow, and, friends:, live, from, central, park, (1999),, the, very, best, of, sheryl, crow, (2003),, live, at, budokan, (2003),, itunes, originals, –, sheryl, crow, (2006), and, hits, &, rarities, (2007)., <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>], [on, november, 7,, 1997,, after, repeating, the, song, ""a, change, would, do, you, <unk>, by, sheryl, crow, for, an, hour,, <unk>, became, soft, adult, contemporary, <unk>, seeking, to, target, the, <unk>, audience, and, citing, the, duplication, of, alternative, music, on, stations, in, other, formats, in, oklahoma, city., <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>]]"
2,foo,"[1.6670092, -0.9686152, 0.774709, -2.0710104]","[1, 0, 1, 0]","[0, 0, 0, 0]","[[films, in, competition, included, babel, by, alejandro, gonzález, iñárritu,, fast, food, nation, by, richard, linklater,, <unk>, by, <unk>, bilge, <unk>, el, <unk>, del, <unk>, by, guillermo, del, toro,, marie, antoinette, by, sofia, coppola,, southland, tales, by, richard, kelly,, volver, by, pedro, <unk>, and, the, wind, that, shakes, the, barley, by, ken, loach, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>], [inspired, by, pedro, almodóvar's, <unk>, kim, began, writing, a, new, screenplay, in, 2008,, finished, its, first, draft, in, 2009,, and, continued, to, edit, it, until, 2013, when, filming, finally, went, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>], [alberto, iglesias, biography,, allmusic, iglesias, has, composed, a, number, of, scores, for, films, directed, by, p

Epoch 0
    Loss:  train = 0.32, valid = 0.36
    Precision 0:  train = 0.71, valid = 0.72
    Recall 0:  train = 0.50, valid = 0.64
    F1 0:  train = 0.59, valid = 0.68
    Precision 1:  train = 0.63, valid = 0.69
    Recall 1:  train = 0.41, valid = 0.38
    F1 1:  train = 0.50, valid = 0.49
    Precision 2:  train = 0.75, valid = 0.76
    Recall 2:  train = 0.56, valid = 0.64
    F1 2:  train = 0.64, valid = 0.69
    Precision 3:  train = 0.75, valid = 0.74
    Recall 3:  train = 0.59, valid = 0.57
    F1 3:  train = 0.67, valid = 0.64
    Precision:  train = 0.71, valid = 0.73
    Recall:  train = 0.52, valid = 0.56
    F1:  train = 0.60, valid = 0.63
Epoch 1
    Loss:  train = 0.29, valid = 0.35
    Precision 0:  train = 0.75, valid = 0.77
    Recall 0:  train = 0.60, valid = 0.60
    F1 0:  train = 0.67, valid = 0.67
    Precision 1:  train = 0.66, valid = 0.68
    Recall 1:  train = 0.44, valid = 0.45
    F1 1:  train = 0.53, valid = 0.54
    Precision 2:  train = 0.75, valid =

,entity,logits,p,gt,sents
0,foo,"[2.2143607, 0.5289778, 3.5724351, 3.3821166]","[1, 1, 1, 1]","[1, 1, 0, 1]","[[cate, blanchett,, willem, dafoe,, jeff, goldblum,, owen, wilson,, bud, cort,, anjelica, huston,, michael, gambon,, bill, murray,, noah, taylor, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>], [these, include, british, performers, paul, scofield, at, the, royal, national, theatre, in, 1980,, anthony, hopkins, in, the, bbc, shakespeare, television, production, (1981),, and, michael, gambon, in, a, stage, production, at, scarborough, directed, by, alan, ayckbourn, in, 1990., <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>], [bad, robots, features, a, host, of, comedy, talent, providing, voices, for, the, machines,, including, lee, kern, as, ""bot"",, michael, gambon,, simon, greenall,, rob, delaney,, terry, mynott,, diane, morgan, and, tom, allen., <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>]]"
1,foo,"[-1.3743584, -2.1815176, -2.3210907, -3.4279535]","[0, 0, 0, 0]","[0, 0, 0, 0]","[[nevertheless,, such, performances, always, require, a, minimum, of, 250, players,, and, have, been, known, to, exceed, 400-pieces,the, sydney, premiere, of, the, return, of, the, king, featured, an, expanded, string, section, (36, violins, divided,, 14, violas,, 11, cellos, and, 9, contrabasses), and, added, woodwind, parts, for, two, cor, anglais,, a, bass, clarinet, and, two, contrabassoons, and, some, of, the, added, brass, instruments, for, the, ride], [andrew, d’angelo, (born, 2, november, 1965, in, greeley,, colorado), is, an, american, jazz, musician, (bass, clarinet,, alto, saxophone,, baritone, saxophone)., <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>], [the, symphony, often, gives, some, of, the, boy, soprano, sections, to, a, young, girl,, and, also, features, a, bass-baritone, part, for, elessar's, oath., <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>]]"
2,foo,"[-0.3990152, -2.2949307, -2.6438944, -3.9865978]","[0, 0, 0, 0]","[0, 0, 0, 0]","[[this, suggests, that, these, neurons, are, producing, a, corollary, discharge,, which, would, allow, for, direct, comparison, of, motor, output, and, auditory, input., <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>], [in, one, study,, speech, therapy, improved, auditory, evoked, potentials, (a, measure, of, brain, activity, in, the, auditory, portions, of, the, brain)., <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>], [au

,entity,logits,p,gt,sents
0,foo,"[-3.205955, -3.6211143, -2.827982, -4.4026413]","[0, 0, 0, 0]","[0, 0, 0, 0]","[[at, one, new, york, show, they, were, discovered, by, an, a&m, records, talent, scout,, patrick, clifford,, and, the, band, signed, their, first, recording, contract., <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>], [although, this, song, entered, the, country, charts,, the, album, itself, was, not, released, due, to, the, closure, of, <unk>, nashville, unit., <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>], [the, song, gained, success, shortly, after, styx, left, wooden, nickel, records, to, move, to, a&m, records, in, 1974, as, it, began, picking, up, airplay, <unk>, at, the, 2010, great, jones, county, fair, eventually, peaking, at, #6, on, the, billboard, hot, 100, in, march, 1975., <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>]]"
1,foo,"[-0.65678114, -2.0061898, 2.7322545, -3.7854004]","[0, 0, 1, 0]","[0, 0, 1, 1]","[[after, discovering, he, is, an, unsuitable, donor, by, dr., leo, <unk>, (chris, <unk>, jack, uses, his, <unk>, clay, aiken,, sara, bareilles,, the, beastie, boys, (mike, d, and, <unk>, mary, j., blige,, elvis, costello,, sheryl, crow,, steve, earle,, wyclef, jean,, norah, jones,, talib, kweli,, cyndi, lauper,, adam, levine,, michael, mcdonald,, rhett, miller,, moby,, robert, randolph,, and, rachael, <unk>, arrange, a, charity, benefit, entitled, <unk>, <unk>], [her, compilation, and, specialty, albums, include, sheryl, crow, and, friends:, live, from, central, park, (1999),, the, very, best, of, sheryl, crow, (2003),, live, at, budokan, (2003),, itunes, originals, –, sheryl, crow, (2006), and, hits, &, rarities, (2007)., <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>], [on, november, 7,, 1997,, after, repeating, the, song, ""a, change, would, do, you, <unk>, by, sheryl, crow, for, an, hour,, <unk>, became, soft, adult, contemporary, <unk>, seeking, to, target, the, <unk>, audience, and, citing, the, duplication, of, alternative, music, on, stations, in, other, formats, in, oklahoma, city., <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>]]"
2,foo,"[0.96162754, -0.62071574, 0.90443325, -2.198689]","[1, 0, 1, 0]","[0, 0, 0, 0]","[[films, in, competition, included, babel, by, alejandro, gonzález, iñárritu,, fast, food, nation, by, richard, linklater,, <unk>, by, <unk>, bilge, <unk>, el, <unk>, del, <unk>, by, guillermo, del, toro,, marie, antoinette, by, sofia, coppola,, southland, tales, by, richard, kelly,, volver, by, pedro, <unk>, and, the, wind, that, shakes, the, barley, by, ken, loach, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>], [inspired, by, pedro, almodóvar's, <unk>, kim, began, writing, a, new, screenplay, in, 2008,, finished, its, first, draft, in, 2009,, and, continued, to, edit, it, until, 2013, when, filming, finally, went, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>], [alberto, iglesias, biography,, allmusic, iglesias, has, composed, a, number, of, scores, for, films, directed, by

...

,entity,logits,p,gt,sents
0,foo,"[-5.442461, -5.561789, -3.0891242, -7.0995717]","[0, 0, 0, 0]","[0, 0, 0, 0]","[[the, 1952, los, angeles, state, diablos, football, team, represented, los, angeles, statecalifornia, state, university,, los, angeles, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>], [barrera, began, playing, college, soccer, at, california, state, university,, los, angeles, in, 2012., <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>], [the, first, chicano, studies, program, was, established, at, california, state, university,, los, angeles, (csula), in, fall, 1968, in, response, to, demands, articulated, by, student, activism, movements., <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>]]"
1,foo,"[-1.7487142, -2.5858564, -3.987946, -3.423605]","[0, 0, 0, 0]","[0, 0, 0, 0]","[[this, was, a, post, that, had, briefly, been, filled, by, darren, o’dea, who, had, moved, on, to, coach, the, under, 18s, with, celtic., <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>], [celtic, defeated, rangers, in, all, five, old, firm, fixtures, that, season., <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>], [next, up,, at, home,, celtic, drew, 1–1, with, manchester, united,, with, scott, mcdonald, putting, celtic, in, the, lead, in, the, 13th, minute,, only, for, ryan, giggs, to, equalise, for, manchester, united, in, the, 84th, minute., <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>]]"
2,foo,"[-3.0717683, -2.7733731, -2.672326, -3.8353672]","[0, 0, 0, 0]","[0, 0, 0, 0]","[[her, disappointment, was,, however,, partly, offset, when, she, won, the, commonwealth, bank, trophy's, best, new, talent, award,, worth, a$5,000,, in, february, 2001., <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>], [total, two-way, merchandise, trade, between, australia, and, finland, in, 2008, was, over, a$2.1, billion, (australia's, 31st, largest, merchandise, trade, partner)., <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>], [the, price, they, paid, per, gram, varied

,entity,logits,p,gt,sents
0,foo,"[-3.4053721, -3.9646344, -3.2143946, -4.018203]","[0, 0, 0, 0]","[0, 0, 0, 0]","[[at, one, new, york, show, they, were, discovered, by, an, a&m, records, talent, scout,, patrick, clifford,, and, the, band, signed, their, first, recording, contract., <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>], [although, this, song, entered, the, country, charts,, the, album, itself, was, not, released, due, to, the, closure, of, <unk>, nashville, unit., <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>], [the, song, gained, success, shortly, after, styx, left, wooden, nickel, records, to, move, to, a&m, records, in, 1974, as, it, began, picking, up, airplay, <unk>, at, the, 2010, great, jones, county, fair, eventually, peaking, at, #6, on, the, billboard, hot, 100, in, march, 1975., <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>]]"
1,foo,"[-2.3706005, -2.3257458, 3.3243847, -3.1847801]","[0, 0, 1, 0]","[0, 0, 1, 1]","[[after, discovering, he, is, an, unsuitable, donor, by, dr., leo, <unk>, (chris, <unk>, jack, uses, his, <unk>, clay, aiken,, sara, bareilles,, the, beastie, boys, (mike, d, and, <unk>, mary, j., blige,, elvis, costello,, sheryl, crow,, steve, earle,, wyclef, jean,, norah, jones,, talib, kweli,, cyndi, lauper,, adam, levine,, michael, mcdonald,, rhett, miller,, moby,, robert, randolph,, and, rachael, <unk>, arrange, a, charity, benefit, entitled, <unk>, <unk>], [her, compilation, and, specialty, albums, include, sheryl, crow, and, friends:, live, from, central, park, (1999),, the, very, best, of, sheryl, crow, (2003),, live, at, budokan, (2003),, itunes, originals, –, sheryl, crow, (2006), and, hits, &, rarities, (2007)., <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>], [on, november, 7,, 1997,, after, repeating, the, song, ""a, change, would, do, you, <unk>, by, sheryl, crow, for, an, hour,, <unk>, became, soft, adult, contemporary, <unk>, seeking, to, target, the, <unk>, audience, and, citing, the, duplication, of, alternative, music, on, stations, in, other, formats, in, oklahoma, city., <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>]]"
2,foo,"[0.5915266, -0.24720621, 1.366611, -2.130046]","[1, 0, 1, 0]","[0, 0, 0, 0]","[[films, in, competition, included, babel, by, alejandro, gonzález, iñárritu,, fast, food, nation, by, richard, linklater,, <unk>, by, <unk>, bilge, <unk>, el, <unk>, del, <unk>, by, guillermo, del, toro,, marie, antoinette, by, sofia, coppola,, southland, tales, by, richard, kelly,, volver, by, pedro, <unk>, and, the, wind, that, shakes, the, barley, by, ken, loach, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>], [inspired, by, pedro, almodóvar's, <unk>, kim, began, writing, a, new, screenplay, in, 2008,, finished, its, first, draft, in, 2009,, and, continued, to, edit, it, until, 2013, when, filming, finally, went, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>], [alberto, iglesias, biography,, allmusic, iglesias, has, composed, a, number, of, scores, for, films, directed, by, p

,entity,logits,p,gt,sents
0,foo,"[-5.056357, -5.211476, -7.2288685, -6.1570387]","[0, 0, 0, 0]","[0, 0, 0, 0]","[[james, mary, magee, (4, september, 1872, in, dublin,, ireland, –, 18, january, 1949, in, county, wicklow)james, magee, cricket, profile, magee, was, capped, in, both, sports,, playing, cricket, for, ireland, and, in, 1896, he, was, part, of, the, british, isles, team, that, toured, south, africa., <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>], [castledermot, is, situated, on, the, main, dublin, -, waterford, road, approximately, ten, km, north, of, carlow, town., <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>], [the, first, was, a, north-south, line, which, would, run, from, swords, to, shanganagh, via, dublin, airport,, finglas,, broadstone,, the, city, centre,, ranelagh,, sandyford, and, cherrywood., <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>]]"
1,foo,"[-0.6578059, -1.208379, -0.7284695, -1.968226]","[0, 0, 0, 0]","[0, 0, 0, 0]","[[barry, gifford, wrote, and, lynch, directed, the, first, and, third, episodes;, lynch, had, previously, adapted, gifford's, wild, at, heart, novel, to, film., <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>], [the, interactive, movie, was, shot, by, the, production, company, propaganda, films,, known, for, filming, commercials, for, at&t, and, coca-cola, and, filmed, 1990's, wild, at, heart., <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>], [another, less-seen, version, of, ""wicked, game"", is, directed, by, david, lynch, and, comprises, scenes, from, the, film, wild, at, heart., "", <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>]]"
2,foo,"[-3.7227688, -2.577091, -3.412148, -2.2585998]","[0, 0, 0, 0]","[0, 0, 0, 0]","[[she, is, obsessed, with, mod, culture, and, music,, as, well, as, 1980s, new, wave,, ska,, madchester, and, 1990s, britpop., <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>], [in, richard, luck's, guide, book, the, madchester, scene:, the, pocket, essential, guide,, luck, described, the, song, as, a, ""breathtaking,, hazy, ode, to, 48, hour, party, people"", and, that, without, the, song, ""we, would, not, be, talking, about, [flowered, up]""., <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>], [the, mild, man, of, rock, ‘n’, roll, manchester, evening, news,, 26, january, 2011, who, be

,entity,logits,p,gt,sents
0,foo,"[-3.434731, -3.615901, -3.0479255, -3.7666955]","[0, 0, 0, 0]","[0, 0, 0, 0]","[[at, one, new, york, show, they, were, discovered, by, an, a&m, records, talent, scout,, patrick, clifford,, and, the, band, signed, their, first, recording, contract., <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>], [although, this, song, entered, the, country, charts,, the, album, itself, was, not, released, due, to, the, closure, of, <unk>, nashville, unit., <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>], [the, song, gained, success, shortly, after, styx, left, wooden, nickel, records, to, move, to, a&m, records, in, 1974, as, it, began, picking, up, airplay, <unk>, at, the, 2010, great, jones, county, fair, eventually, peaking, at, #6, on, the, billboard, hot, 100, in, march, 1975., <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>]]"
1,foo,"[-2.640361, -1.4340609, 4.3781366, -2.4032195]","[0, 0, 1, 0]","[0, 0, 1, 1]","[[after, discovering, he, is, an, unsuitable, donor, by, dr., leo, <unk>, (chris, <unk>, jack, uses, his, <unk>, clay, aiken,, sara, bareilles,, the, beastie, boys, (mike, d, and, <unk>, mary, j., blige,, elvis, costello,, sheryl, crow,, steve, earle,, wyclef, jean,, norah, jones,, talib, kweli,, cyndi, lauper,, adam, levine,, michael, mcdonald,, rhett, miller,, moby,, robert, randolph,, and, rachael, <unk>, arrange, a, charity, benefit, entitled, <unk>, <unk>], [her, compilation, and, specialty, albums, include, sheryl, crow, and, friends:, live, from, central, park, (1999),, the, very, best, of, sheryl, crow, (2003),, live, at, budokan, (2003),, itunes, originals, –, sheryl, crow, (2006), and, hits, &, rarities, (2007)., <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>], [on, november, 7,, 1997,, after, repeating, the, song, ""a, change, would, do, you, <unk>, by, sheryl, crow, for, an, hour,, <unk>, became, soft, adult, contemporary, <unk>, seeking, to, target, the, <unk>, audience, and, citing, the, duplication, of, alternative, music, on, stations, in, other, formats, in, oklahoma, city., <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>]]"
2,foo,"[0.28190494, 0.06448352, 1.5640664, -1.8771955]","[1, 1, 1, 0]","[0, 0, 0, 0]","[[films, in, competition, included, babel, by, alejandro, gonzález, iñárritu,, fast, food, nation, by, richard, linklater,, <unk>, by, <unk>, bilge, <unk>, el, <unk>, del, <unk>, by, guillermo, del, toro,, marie, antoinette, by, sofia, coppola,, southland, tales, by, richard, kelly,, volver, by, pedro, <unk>, and, the, wind, that, shakes, the, barley, by, ken, loach, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>], [inspired, by, pedro, almodóvar's, <unk>, kim, began, writing, a, new, screenplay, in, 2008,, finished, its, first, draft, in, 2009,, and, continued, to, edit, it, until, 2013, when, filming, finally, went, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>, <unk>], [alberto, iglesias, biography,, allmusic, iglesias, has, composed, a, number, of, scores, for, films, directed, by, p

KeyboardInterrupt: 

# 4 Test

## 4.1 Define test data

In [9]:
test_data = [
    {
        'ent': 1000,
        'classes': [1, 0, 1, 0],  # married, male, American, actor
        'sents': [
            'Michelle is married',
            'Michelle is female',
            'Michelle is American'
        ]
    },
    {
        'ent': 2000,
        'classes': [1, 0, 0, 0],  # married, male, American, actor
        'sents': [
            'Angela is married',
            'Angela is female',
            'Angela is German'
        ]
    }
]

test_set = [Sample(
    item['ent'],
    item['classes'],
    [[vocab[word] for word in tokenize(sent)] for sent in item['sents']]
) for item in test_data]

test_loader = DataLoader(test_set, batch_size=len(test_set), collate_fn=generate_batch)

## 4.2 Forward test batch

In [10]:
test_loss = 0.0
with torch.no_grad():
    for sents_batch, classes_batch in test_loader:
        logits_batch = classifier(sents_batch)

        print(logits_batch)

        loss = criterion(logits_batch, classes_batch.float())
        test_loss += loss.item()

test_loss /= len(test_loader)

print(f'Test loss = {test_loss}')

tensor([[-2.1373, -1.8922, -2.0493, -1.3927],
        [-2.1373, -1.8922, -2.0493, -1.3927]])
Test loss = 0.9392150044441223
